In [1]:
import pickle
import numpy as np
#import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
import torch
DATA_PATH = "D:\\Synology Backup\\mhunterklein\\Projects\\lpne\\DATA\\Pickle_Files_For_UMC\\ANX_EPM_data_2022_March_2.pkl"
OLD_DATA_PATH = "D:\\Synology Backup\\mhunterklein\\Projects\\lpne\\DATA\\Dalton\\Directed_Spectrum_Data\\EPM_DS_2021_11_29.mat"
UMC_PATH = ".\\Universal-Mouse-Code\\"
LPNE_FOLDER = "D:\\Feature Pipeline Demo\\lpne-data-analysis"
AUSTIN_CODE = "..\\ForHunter\\"
dCSFA_NMF_PATH = "C:\\Users\\micha\\Documents\\cpne\\lpne-missing-data\\models\\dCSFA_NMF.py"

EPM_TRAIN_DICT_PATH = ".\\EPM_model\\EPM_train_dict_hand_picked.pkl"
EPM_VAL_DICT_PATH = ".\\EPM_model\\EPM_val_dict_hand_picked.pkl"
OFT_TRAIN_FILE_NAME = ".\\OFT_model\\OFT_train_dict_old_features_hand_picked.pkl"
OFT_VAL_FILE_NAME = ".\\OFT_model\\OFT_validation_dict_old_features_hand_picked.pkl"
FLX_TRAIN_FILE = ".\\FLX_model\\FLX_train_dict_old_features.pkl"
FLX_VAL_FILE = ".\\FLX_model\\FLX_validation_dict_old_features.pkl"

FEATURE_VECTOR = ["X_power_1_2","X_coh_1_2","X_gc_1_2"]
EPM_DISCOVERY_DICT = {}
sys.path.append(DATA_PATH)
sys.path.append(OLD_DATA_PATH)
sys.path.append(LPNE_FOLDER)
sys.path.append(dCSFA_NMF_PATH)
sys.path.append(UMC_PATH)
#from dCSFA_model import dCSFA_model
import umc_data_tools as umc_dt
#from dCSFA_NMF import dCSFA_NMF
#import data_tools
#import ml_base, nmf_elastic, nmf_log_ri

#For Consistency
RANDOM_STATE=42

In [2]:
#We aren't working with the test data yet
with open(EPM_TRAIN_DICT_PATH,"rb") as f:
    train_dict = pickle.load(f)

with open(EPM_VAL_DICT_PATH,"rb") as f:
    validation_dict = pickle.load(f)

#scale_vector = np.array([np.arange(1,57) for feature in range(64)]).flatten()
#Load the data
running_idx = 0
feature_groups = []
for idx,feature in enumerate(FEATURE_VECTOR):
    f_begin = running_idx
    f_end = f_begin + train_dict[feature].shape[1] 
    if idx == 0:
        f_end = f_end -1
    feature_groups.append((f_begin,f_end))

    running_idx = f_end

X_train = np.hstack([train_dict[feature] for feature in FEATURE_VECTOR])
y_roi_train = ~(train_dict['y_ROI']%2).astype(bool)
y_hc_train = train_dict['y_Homecage'].astype(bool)
y_vel_train = np.array(train_dict['y_vel'])
y_mouse_train = train_dict['y_mouse']
y_time_train = train_dict['y_time']

train_roi_nan_mask = (train_dict['y_ROI'] > 0)
train_in_hc_mask = y_hc_train
train_in_task_mask = ~y_hc_train
train_velocity_gt_8_mask = np.ones_like(y_time_train)#(y_vel_train > 0)

train_with_hc_mask = train_velocity_gt_8_mask
#train_with_hc_mask = np.logical_and(train_roi_nan_mask,train_velocity_gt_8_mask)
train_in_task_mask = np.logical_and(train_with_hc_mask,train_in_task_mask)


X_task_train = X_train[train_with_hc_mask==1]
y_hc_task_train = ~y_hc_train[train_with_hc_mask==1]
y_mouse_task_train = y_mouse_train[train_with_hc_mask==1]
y_roi_task_train = y_roi_train[train_with_hc_mask==1]

X_validation = np.hstack([validation_dict[feature] for feature in FEATURE_VECTOR])
y_roi_validation = ~(validation_dict['y_ROI']%2).astype(bool)
y_hc_validation = validation_dict['y_Homecage'].astype(bool)
y_vel_validation = validation_dict['y_vel']
y_mouse_validation = validation_dict['y_mouse']
y_time_validation = validation_dict['y_time']
validation_roi_nan_mask = (validation_dict['y_ROI'] > 0)
validation_in_task_mask = ~y_hc_validation
validation_in_hc_mask = y_hc_validation
validation_velocity_gt_8_mask = np.ones_like(y_time_validation)#(y_vel_validation > 0)

validation_with_hc_mask = np.logical_and(validation_roi_nan_mask,validation_velocity_gt_8_mask)
validation_with_hc_mask = validation_velocity_gt_8_mask
validation_in_task_mask = np.logical_and(validation_with_hc_mask,validation_in_task_mask)

X_task_val = X_validation[validation_with_hc_mask==1]
y_hc_task_val = ~y_hc_validation[validation_with_hc_mask==1]
y_mouse_task_val = y_mouse_validation[validation_with_hc_mask==1]
y_roi_task_val = y_roi_validation[validation_with_hc_mask==1]

intercept_mask = OneHotEncoder().fit_transform(y_mouse_task_train.reshape(-1,1)).todense()

In [3]:
N_COMPONENTS=20
DIM_IN = X_task_train.shape[1]
'''model = dCSFA_NMF(n_components=N_COMPONENTS,dim_in=DIM_IN,device='auto',n_intercepts=intercept_mask.shape[1],
                n_sup_networks=1,optim_name='AdamW',recon_loss='IS',sup_recon_weight=1,sup_weight=1,
                useDeepEnc=True,h=256,sup_recon_type="Residual",feature_groups=feature_groups)
model.fit(X_task_train,y_hc_task_train,intercept_mask=intercept_mask,batch_size=128,lr=1e-3,
        n_pre_epochs=100,n_epochs=3000,verbose=True,pretrain=True,best_model_name="DS_EPM_Network_March_25_state_dict.pt")
torch.save(model,"DS_EPM_Network_HC_vs_Task_March_25.pt")'''
model = torch.load("EPM_Network_HC_vs_Task_March_25.pt",map_location="cpu")
model.device="cpu"

In [7]:
y_pred, s = model.predict(X_task_train,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_train,y_mouse_task_train)
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_train,y_mouse_task_train,s,True)
print("train roc auc dict: ",roc_auc_dict)
print("train mw auc dict: ",mw_auc_dict)

y_pred, s = model.predict(X_task_val,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_val,y_mouse_task_val)
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_val,y_mouse_task_val,s,True)
print("validation roc auc dict: ",roc_auc_dict)
print("validation mw auc dict: ",mw_auc_dict)


train roc auc dict:  {'auc_method': 'sklearn_roc_auc', 'Mouse04201': 0.9697452229299364, 'Mouse04202': 1.0, 'Mouse04205': 0.8388888888888889, 'Mouse04215': 1.0, 'Mouse0630': 0.9966832504145937, 'Mouse0634': 0.9973684210526316, 'Mouse0643': 1.0, 'Mouse1551': 0.9946401225114854, 'Mouse39114': 1.0, 'Mouse39124': 1.0, 'Mouse39133': 1.0, 'Mouse6291': 0.9983818770226538, 'Mouse6292': 1.0, 'Mouse6293': 0.9854838709677419, 'Mouse69064': 1.0, 'Mouse69074': 1.0, 'Mouse8580': 0.994535519125683, 'Mouse8581': 1.0, 'Mouse8582': 0.8690476190476191, 'Mouse8891': 0.9950166112956811, 'Mouse8894': 0.7096774193548387}
train mw auc dict:  {'auc_method': 'mannWhitneyU', 'Mouse04201': (0.0, 1.948546217773956e-107), 'Mouse04202': (0.0, 1.2234124412320264e-109), 'Mouse04205': (0.0, 8.504586546833693e-07), 'Mouse04215': (0.0, 4.406077578101495e-108), 'Mouse0630': (0.0, 5.695529992412083e-84), 'Mouse0634': (0.0, 7.095752964106866e-68), 'Mouse0643': (0.0, 1.062145166618735e-49), 'Mouse1551': (0.0, 2.0725984242173

In [4]:
y_pred, s = model.predict(X_task_train,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_train,y_mouse_task_train)
print("train roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_train,y_mouse_task_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
y_pred, s = model.predict(X_task_val,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_val,y_mouse_task_val)
print("val roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_val,y_mouse_task_val,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))

#umc_dt.make_recon_plots(model,X_train,0,task="EPM (HC vs Task) Training",saveFile="epm_train_recon.png")
#umc_dt.make_recon_plots(model,X_validation,0,task="EPM (HC vs Task) Val",saveFile="epm_val_recon.png")

train roc auc mean: 0.97 +/- 0.02
train mw auc mean: 0.00 +/- 0.00
val roc auc mean: 0.71 +/- 0.03
val mw auc mean: 0.20 +/- 0.02


In [29]:
import pandas as pd
def make_projection_csv(pickle_file,model,X_feature_list,other_features,save_file):

    with open(pickle_file,'rb') as f:
        project_dict = pickle.load(f)
    
    X_project = np.hstack([project_dict[feature] for feature in X_feature_list])
    s = model.transform(X_project)[3]
    s = s[:,0]
    save_dict = {}
    save_dict['scores'] = s
    for feature in other_features:
        save_dict[feature] = project_dict[feature]

    df = pd.DataFrame.from_dict(save_dict)
    df.to_csv(save_file,index=False,header=True)

    return df

df = make_projection_csv(EPM_TRAIN_DICT_PATH,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = r".\\EPM_Train_Projection_2022_March_25.csv")

df = make_projection_csv(EPM_VAL_DICT_PATH,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = r".\\EPM_Val_Projection_2022_March_25.csv")

### OFT Projections

In [9]:
with open(OFT_TRAIN_FILE_NAME,'rb') as f:
    train_dict = pickle.load(f)

with open(OFT_VAL_FILE_NAME,'rb') as f:
    val_dict = pickle.load(f)

NUM_FREQS = 56
NUM_FEATURES = np.hstack([train_dict[feature] for feature in FEATURE_VECTOR]).shape[1] // NUM_FREQS

#Train Arrays
oft_X_train = np.hstack([train_dict[feature] for feature in FEATURE_VECTOR])
oft_y_hc_train = train_dict['y_Homecage'].astype(bool)
oft_y_task_train = ~oft_y_hc_train
oft_y_ROI_train = train_dict['y_ROI']
oft_y_vel_train = train_dict['y_vel']
oft_y_mouse_train = train_dict['y_mouse']
oft_y_time_train = train_dict['y_time']

#Validation Arrays
oft_X_val = np.hstack([val_dict[feature] for feature in FEATURE_VECTOR])
oft_y_hc_val = val_dict['y_Homecage'].astype(bool)
oft_y_task_val = ~oft_y_hc_val
oft_y_ROI_val = val_dict['y_ROI']
oft_y_vel_val = val_dict['y_vel']
oft_y_mouse_val = val_dict['y_mouse']
oft_y_time_val = val_dict['y_time']

oft_X = np.vstack([oft_X_train,oft_X_val])
oft_y_task = np.hstack([oft_y_task_train,oft_y_task_val])
oft_y_mouse = np.hstack([oft_y_mouse_train,oft_y_mouse_val])

In [10]:
y_pred, s = model.predict(oft_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse)
print("ROC AUCs: ",roc_auc_dict)
mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("MW AUCs: ",mw_auc_dict)

ROC AUCs:  {'auc_method': 'sklearn_roc_auc', 'Mouse04191': 0.6398373677671949, 'Mouse04193': 0.7446073993602511, 'Mouse04201': 0.6934355059039278, 'Mouse04202': 0.7788481079543146, 'Mouse04205': 0.7973735095069288, 'Mouse04215': nan, 'Mouse3191': 0.46943803969438047, 'Mouse3192': nan, 'Mouse3193': 0.5603413815158298, 'Mouse3194': nan, 'Mouse3203': nan, 'Mouse39114': 0.6398191823899371, 'Mouse39124': 0.7493036735922914, 'Mouse39125': 0.7240263667150459, 'Mouse39133': 0.7559411914873866, 'Mouse69064': 0.5985302123026675, 'Mouse69065': 0.6784348864994026, 'Mouse69072': 0.5974302013054686, 'Mouse69074': 0.6164171122994653}
Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC

C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


In [6]:
y_pred, s = model.predict(oft_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("EPM->FLX roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("EPM->FLX mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,oft_X,0,task="EPM onto OFT (HC vs Task) Train and Val",saveFile="epm_onto_oft_recon.png")

Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC cannot be calculated
n_positive samples  500
n_negative samples  0
EPM->FLX roc-auc: 0.6695856092196328 +/- 0.023721198086133696
EPM->FLX mw-auc: 0.23 +/- 0.03


C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


In [34]:
df = make_projection_csv(OFT_TRAIN_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = r".\\EPM_onto_OFT_HC_vs_Task_Train_Projection_2022_March_25.csv")

df = make_projection_csv(OFT_VAL_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = r".\\EPM_onto_OFT_HC_vs_Task_Val_Projection_2022_March_25.csv")

### FLX Projections

In [11]:
with open(FLX_TRAIN_FILE,"rb") as f:
    flx_train_dict = pickle.load(f)

with open(FLX_VAL_FILE,"rb") as f:
    flx_validation_dict = pickle.load(f)
    
flx_X_train = np.hstack([flx_train_dict[feature] for feature in FEATURE_VECTOR])
flx_y_train = flx_train_dict['y_flx']
flx_y_mouse_train = flx_train_dict['y_mouse']
flx_y_expDate_train = flx_train_dict['y_expDate']
flx_y_time_train = flx_train_dict['y_time']

flx_X_validation = np.hstack([flx_validation_dict[feature] for feature in FEATURE_VECTOR])
flx_y_validation = flx_validation_dict['y_flx']
flx_y_mouse_validation = flx_validation_dict['y_mouse']
flx_y_expDate_validation = flx_validation_dict['y_expDate']
flx_y_time_validation = flx_validation_dict['y_time']

flx_X = np.vstack([flx_X_train,flx_X_validation])
flx_y_task = np.hstack([flx_y_train,flx_y_validation])
flx_y_mouse = np.hstack([flx_y_mouse_train,flx_y_mouse_validation])
flx_y_expDate = np.hstack([flx_y_expDate_train,flx_y_expDate_validation])
flx_y_time = np.hstack([flx_y_time_train,flx_y_time_validation])

In [12]:
y_pred, s = model.predict(flx_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse)
print("ROC AUCs: ",roc_auc_dict)
mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse,s,True)
print("MW AUCs: ",mw_auc_dict)

ROC AUCs:  {'auc_method': 'sklearn_roc_auc', 'Mouse3191': 0.45662683872775905, 'Mouse3192': 0.6928735167206042, 'Mouse3193': 0.43572469475749065, 'Mouse3194': 0.44847281262286876, 'Mouse3202': nan, 'Mouse3203': 0.5463417444334471, 'Mouse99003': nan}
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
MW AUCs:  {'auc_method': 'mannWhitneyU', 'Mouse3191': (0.5646683415872336, 1.0700018576610867e-11), 'Mouse3192': (0.18093710159850937, 8.128284515007517e-240), 'Mouse3193': (0.617558568880732, 2.006461702164672e-34), 'Mouse3194': (0.599350237282302, 2.2891940457461176e-25), 'Mouse3203': (0.43530432227916166, 1.9584289796246316e-11)}


C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


In [9]:
y_pred, s = model.predict(flx_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_task,flx_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_task,flx_y_mouse,s,True)
print("EPM->FLX roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("EPM->FLX mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,flx_X,0,task="EPM onto FLX (FLX vs Saline) Train and Val",saveFile="epm_onto_flx_recon.png")

Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
EPM->FLX roc-auc: 0.5160079214524339 +/- 0.04833946628011897
EPM->FLX mw-auc: 0.48 +/- 0.08


C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "
C:\Users\micha\anaconda3\envs\lpne\lib\site-packages\sklearn\metrics\_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


In [42]:
umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse,s,True)

Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0


{'auc_method': 'mannWhitneyU',
 'Mouse3191': (0.5646648088276596, 1.0727617945997764e-11),
 'Mouse3192': (0.18092989972540943, 7.92991562462114e-240),
 'Mouse3193': (0.6175495242040148, 2.0298596107301814e-34),
 'Mouse3194': (0.5993544200476357, 2.278683853919311e-25),
 'Mouse3203': (0.435291589324318, 1.940787661371215e-11)}

In [41]:
df = make_projection_csv(FLX_TRAIN_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_flx'],
                    save_file = r".\\EPM_onto_FLX_flx_vs_saline_Train_Projection_2022_March_25.csv")

df = make_projection_csv(FLX_VAL_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_flx'],
                    save_file = r".\\EPM_onto_FLX_flx_vs_saline_Val_Projection_2022_March_25.csv")